## MNIST dataset classification using SGD and TensorFlow
### This code is taken Fundamentals of Deep Learning by Nikhil Buduma

In [1]:
import tensorflow as tf
tf.__version__

/home/harshal/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/harshal/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/harshal/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/harshal/anaconda3/envs/tf/lib/python3.7/site-pack

'1.14.0'

4 step:
* Inference 
* Loss
* Train
* Evaluate

In [2]:
def inference(x):
    w_init = tf.constant_initializer()
    w = tf.get_variable("w",[784,10],
                        tf.float32,initializer=w_init)
    b_init = tf.constant_initializer()
    b = tf.get_variable("b",[10],tf.float32,
                        initializer=b_init)
    output = tf.nn.softmax(tf.matmul(x,w) + b)
    return output

In [3]:
def loss(output,y):
    dot_matrix = y*tf.log(output)
    xentropy = - tf.reduce_sum(dot_matrix)
    loss = tf.reduce_mean(xentropy)
    return loss

In [4]:
def training(cost, global_step):
    tf.summary.scalar("cost",cost)
    optimize = tf.train.GradientDescentOptimizer(
                    learning_rate)
    train_op = optimize.minimize(cost,
                        global_step=global_step)
    return train_op

In [5]:
def evaluate(output,y):
    correct_prediction = tf.equal(tf.arg_max(output,1),
                               tf.arg_max(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,
                                tf.float32))
    return accuracy

## Loading data

In [6]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [10]:
# Parameters
learning_rate = 0.01
training_epoch = 100
batch_size = 10
display_step = 1

with tf.Graph().as_default():
    x = tf.placeholder(tf.float32,[None,784])
    y = tf.placeholder(tf.float32,[None,10])
    output = inference(x)
    cost = loss(output,y)
    global_step = tf.Variable(0,name="global_step",
                             trainable = False)
    train_op = training(cost,global_step)
    eval_op = evaluate(output,y)
    summary_op = tf.summary.merge_all()
    saver = tf.train.Saver()
    sess = tf.Session()
    summary_writter = tf.summary.FileWriter(
                        "logs/",
                         graph_def=sess.graph_def)
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    
    for epoch in range(training_epoch):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/
                          batch_size)
        for i in range(total_batch):
            mbatch_x, mbatch_y = mnist.train.next_batch(
                                batch_size)
            feed_dict = {x:mbatch_x,
                         y:mbatch_y}
            sess.run(train_op,feed_dict=feed_dict)
            minibatch_cost = sess.run(cost,
                                     feed_dict=feed_dict)
            avg_cost +=minibatch_cost/total_batch
        if epoch % display_step == 0:
            val_feed_dict = {
                            x:mnist.validation.images,
                            y:mnist.validation.labels 
                            }
            accuracy = sess.run(eval_op,
                                feed_dict=val_feed_dict)
            print(f"Validation Error: {1 - accuracy}")
            summary_str = sess.run(summary_op,
                                  feed_dict=feed_dict)
            summary_writter.add_summary(
                            summary_str,
                            sess.run(global_step))
            saver.save(sess,"log/model-checlpoint",
                        global_step=global_step)
        
        print("Optimization Finished!")
        test_feed_dict = {
                          x:mnist.test.images,
                          y:mnist.test.labels
                          }
        accuracy = sess.run(eval_op,feed_dict=test_feed_dict)
        print(f"Test Accuracy: {accuracy}")

Validation Error: 0.0835999846458435
Optimization Finished!
Test Accuracy: 0.9162999987602234
Validation Error: 0.07959997653961182
Optimization Finished!
Test Accuracy: 0.9160000085830688
Validation Error: 0.08020001649856567
Optimization Finished!
Test Accuracy: 0.9169999957084656
Validation Error: 0.071399986743927
Optimization Finished!
Test Accuracy: 0.9217000007629395
Validation Error: 0.07679998874664307
Optimization Finished!
Test Accuracy: 0.9207000136375427
Validation Error: 0.0756000280380249
Instructions for updating:
Use standard file APIs to delete files with this prefix.
Optimization Finished!
Test Accuracy: 0.9236999750137329
Validation Error: 0.08459997177124023
Optimization Finished!
Test Accuracy: 0.914900004863739
Validation Error: 0.0756000280380249
Optimization Finished!
Test Accuracy: 0.9239000082015991
Validation Error: 0.07740002870559692
Optimization Finished!
Test Accuracy: 0.9203000068664551
Validation Error: 0.07580000162124634
Optimization Finished!
Test A

Validation Error: 0.07620000839233398
Optimization Finished!
Test Accuracy: 0.9233999848365784
Validation Error: 0.07440000772476196
Optimization Finished!
Test Accuracy: 0.921999990940094
Validation Error: 0.07639998197555542
Optimization Finished!
Test Accuracy: 0.9185000061988831
Validation Error: 0.0812000036239624
Optimization Finished!
Test Accuracy: 0.921999990940094
Validation Error: 0.07700002193450928
Optimization Finished!
Test Accuracy: 0.9178000092506409
Validation Error: 0.07859998941421509
Optimization Finished!
Test Accuracy: 0.920199990272522
Validation Error: 0.07440000772476196
Optimization Finished!
Test Accuracy: 0.9223999977111816
Validation Error: 0.08079999685287476
Optimization Finished!
Test Accuracy: 0.9193999767303467
Validation Error: 0.07499998807907104
Optimization Finished!
Test Accuracy: 0.9232000112533569
Validation Error: 0.07899999618530273
Optimization Finished!
Test Accuracy: 0.9192000031471252
Validation Error: 0.07660001516342163
Optimization Fin

In [11]:
import tensorboard

In [13]:
!tensorboard --logdir="logs/"

/home/harshal/anaconda3/envs/tf/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/harshal/anaconda3/envs/tf/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/harshal/anaconda3/envs/tf/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/harshal/anaconda3/envs/tf/li